In [14]:
!pip install duckdb
!pip install notebook
!pip install jupysql pandas matplotlib duckdb-engine

You should consider upgrading via the '/Users/jarred.luzada/.asdf/installs/python/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/jarred.luzada/.asdf/installs/python/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/jarred.luzada/.asdf/installs/python/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [15]:
import os
osm_dataset = "cemetery"
input_path = f"./data/geojson/{osm_dataset}.geojson"
input_filename = os.path.basename(input_path)

In [16]:
import duckdb
from sqlalchemy.engine import create_engine

%load_ext sql
conn = create_engine('duckdb:///data/local.duckdb')
%sql conn --alias duckdb
%config SqlMagic.named_parameters="enabled" 

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [17]:
%%sql
INSTALL psql FROM community;
INSTALL spatial;
LOAD spatial;
LOAD psql;


Running query in 'duckdb'

Success


In [18]:
%%sql
CREATE OR REPLACE TABLE point_data AS
SELECT *
FROM ST_Read("{{input_path}}")

Running query in 'duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Count


In [19]:

import random
import string

try:
    conn.raw_connection().remove_function('random_string')
except:
    pass

# Define the Python function to generate a random string of length 'n'
def random_string(n: int) -> str:
    return ''.join(random.choices(string.ascii_letters + string.digits, k=n))

# Register the function with DuckDB

conn.raw_connection().create_function('random_string', random_string, side_effects=True)

In [20]:
%sql SELECT random_string(10) AS random_string;

Running query in 'duckdb'

random_string
tBHRD1FixH


In [21]:

%%sql
CREATE OR REPLACE TABLE point_data_centroid AS
WITH centroid AS (
    FROM point_data
    SELECT
        rowid,
        centroid: geom.ST_Centroid(),
    ORDER BY hash(rowid + 43) DESC
)
FROM centroid
SELECT
    id: ROW_NUMBER() OVER (ORDER BY centroid),
    location: 'point("' || centroid.ST_X() || ',' || centroid.ST_Y() || '")',
    description: random_string(250)
ORDER BY hash(rowid + 43)

Running query in 'duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Count


In [22]:
%sql SELECT * FROM point_data_centroid

Running query in 'duckdb'

id,location,description
117886,"point(""-85.6466748899667,39.28365177477235"")",t3uwaQZtslUR99gZ12P9G2rhWw1mBN61cruEeQZVHFXXAzsgEfGhjlbPWaeuQxb6cmvHCpkNpiBJs6rHddHYGHlYzA29wHp82NbgJYaEu1YCKI4bVEjS9CsPDxXwEuhw8hTqtJythUT2wku97P6oTax3AOgA6O2zA2uWBh1BY8NQVamQfQ40lcWpmTwjiTGoHyEH65K67UMkn7vqq83Smo7zdzB4xNB48eD7jMI05sVhCB3doI1jHdKU7c
123815,"point(""139.0385562473188,36.42874679055316"")",mSEBLha9A404SKDOQIxDByU6bUMXFR0qVoD6XihzncrBqTYr8GnIPAZCKPnBGyJ5kf9PoYXY27C6Pq8fCNh5MNZHg3iMxDl1inoGpezd7nWwbZvul2MxMT7xFppMgH7botjOBunyfzjV6E7pDDddI1fhc9KdI7HVTSCQc7rXihnmtPmIQPccOCJqJ70MsKus4plKxB9RdU1t3P3cdHWszMrYtbeviINJ2cuoffaNRNStKsABThSueH11EU
24588,"point(""99.10831835077646,11.923422147551403"")",fd8i0BWy8Or4D7AdqAHS03I63NRaYxO2yO3LsseaI7TrnbF404CKmJCvna45iAXLwR6FyRdZHKsmpD86gnhjewc79OuOh5lAicN0Pbc0sfTCxOWAxhC3wXAKOwwUhsmMsUSqfm7wYt8c74uVoiJaoLVBWQvpZHpKtaypWS0xc1NPiKEeTlHOL2b3yhBaosMIXBeV8SHJXLUNpfecX8dGJ2Srd1qf4FbP3RBoaFhBVWcdOeb7nXD6zfrmGB
398352,"point(""37.69955308014503,54.3793027489593"")",fSbMhZ6u9qaHxRTMyu3BMAaIymU2niUOG0mCuVfmCQ8RwLkLe58F8Z9HhJw65NNZvujBdm5By15uatYIfLLECl3WWzpthiNQuBSDKivf4IplBD9nF4DOUntxHsoMqGkW9nHh0eyQdIGRiZwE8XgPukwKpwXa4t8wnU4faSVBblhKLtJnNnLqao33mSEqrZzqjmJ0Fo7bXEpABpT9jq1oVt1rOws5pwT84f6DtO05i1RPdrEMnUsx5uBUiQ
80284,"point(""-8.084735554647613,5.9921804183259475"")",uw9fkvXbWRUQTNfbi9QRFoeaSBagpR8b6hCopSP4zChCTefOFMVZfdFbA55YjHnjvegwbp5mlvn4aEKsbMe2vzbGLNDbXXBgMWlZFJZbd4rADJr6SOzFszh3JSWuxmXuBTU0JA8W4uDoxCzDbEgmfYEzfFpMiGFdLxx9SZ1kK43Mz1vaR3KEN8s4AlXOBUcUulTtFZ6FQm6M97suJCyu6lWEDzdc8DJPeZGR6zD2gCbZfTVIAL7YJdzzd4
113274,"point(""139.44185915948967,35.90150919549181"")",AjRTbjtigCQziXJ9V27qdrjvEKLIgzH5L8mYmDwbunOOY0XQPXQdC7UsuEGuYOSOXLCUHjLYaHXjoF9P7V8VaviBXuQJQ9HMKCEjweSzOZ1pELSmyF8EniKXyCPocGZvMd8msPhA5zzYNEtmfZpM7B62V8xxHMhGCWVGHeXIUi6zHgtFAbonQjDhjvueVrboEpQrVOdSHsIbT73j6cjvf1r5tzy6NGG5IRYZQae8UjBDz7M3SmLF4YjwNN
91013,"point(""36.06921455300512,50.51046800739962"")",BI5SwHelLRfsUk0rnmK8YvBGnqJYcRcWDFOXOO91ZGIx0M0iFdpffIQtcHaJN5YW5ZTUnWoyoniKS9AfTjLKIIzAC4NfHqtGAzyfJTerXF2NuT98oIh0o8UuGl7WAuhWnZdDrDllqEuuoayK12kaiOYCJuFsIsCWAdrYzwBtLCVlmKd60PIujpq4pxv82iR5p6HZUBc0yjgpSbnAjKg7mMdT0Mgz68RKohGUachbIlIseSVxQ1FIGFpgHY
238267,"point(""-117.28437550489467,34.114246342027826"")",YFKgEASo7s1YWzG0ZHnj5132KgMabYiXWXMH9fg0sEJg74NnZgO6HlNlfblWAlerauKwbZVZK2ggK9gOjRgilFW93oNcBvoeFDqkvSrP0qNVKe6Vxs6BIGN0YniB0cI7ZbdvOU6Xxvggc8yfg8SwenyQU0ipIp71HD3uuq4BqQj7WIYMeHpDTK5iBd1rAF5p9mVagWomd24YlAgW80VpK68peyMUuQVeN66ZPbiC2AnWiIwTZfmdUNVaGY
130264,"point(""-83.69704215677237,41.71782106521608"")",zeDEIKaQBWAktg2WlogthPamICSVZiju4AUPL9IOXoXr0A6hczI18V4ohMomFvsEbnXwvDLf2uz0wzuV9JL3Ini8G0BfRjCIYBub5OPLuu49Xe33r0ru3lysEqPXb3hC2Js9vPmKIEId8D3G5N0bSg6UOtFRGjd8Wk4WRmQUa7O08iVnnqvx1lx49iZRMaMyhS3psWimZLpsSH3EvHfy3egZbNVEIIPUbfnuGGR5Seu5c2wHjzaG48p0Il
13305,"point(""-70.88601036685225,18.564127708683035"")",91RliTSGgMWbosZT55YvJHhAgdEAsF1pxffKH7QJxqWDQUWW0Pewox4yYg7T6DmzPnOJ7jV9VdKws03UGnqTZwTlGBlJrj1tPeWXEeV9A99REuBshgzBZ6Y69lDKwTgzU6ZgHywkncVtXIos8d611k4z0af9yFEE7iS2C6cThnTmoURmS72b2M0rmIgL7lKPvPSQGmxbXz6KGSeGGTakPNN7oQIDtqYypkaMivcSCLmI5KQCSw71zJVmGO


In [23]:
%%sql
SELECT COUNT(*) FROM point_data_centroid as data;

Running query in 'duckdb'

count_star()
402325


In [24]:
%%sql
SUMMARIZE point_data_centroid

Running query in 'duckdb'

column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
id,BIGINT,1,402325,377825,201163.0,116141.36786333608,100625,201065,301677,402325,0.00
location,VARCHAR,"point(""-0.00022686332641463965,44.553858467915205"")","point(""99.99367990063318,9.730698042662416"")",327363,None,None,None,None,None,402325,0.00
description,VARCHAR,000G0Y1LdishZz3EWQLRvJCZNrsekMRMn6nGq30xpNh0zBazfnSRAHRCNwFVm8A6GKTlk6pYKNUsK5FxCJt4Qwzx2DbtmJEaKLIPcyWXxBOTes66AvAkmsxacVflruJGBium2bVQ9XQfCdaloA6CDKjURBme1InbFWfxeRX0d01WI3BY0UwM9HSZToEp06fdGJqEh6nzHdPskI0rD2OFvUtEC1Ofxorr721pFtc68yI1yURoBGmNkkv1Db,zzz4Ed6tRtFE4wU3LUyqXbD0xEssMul4Y3etRh3Vjn9Ko7s6BuJ2aIw7OFGm0s6lwyb63gBNu9cvRIlOIGnDwLgkp4Ppxet10eYJlZTz0vtJdDS52ewJadEnBmFSHv3E7tbQIfwh9X3pfw6FReiOLnMcuCmLh0xbmAUrwgHRRWrbX1EeIBsag8gBgvtuGzn3uCc2ldF3QZbtzIhD8YJgIenrDsTD6MpFFMJPcPSkinGwc336TJ1KULuVIO,408918,None,None,None,None,None,402325,0.00


In [25]:
export_filename = input_filename.replace(".geojson", ".json")
export_path = "./data/json/" + export_filename

os.makedirs(os.path.dirname(export_path), exist_ok=True)

print(f"Exporting to {export_filename}")
print(f"Exporting to {export_path}")

%sql COPY (SELECT * FROM point_data_centroid) TO '{{export_path}}' (FORMAT JSON);

Exporting to cemetery.json
Exporting to ./data/json/cemetery.json


Running query in 'duckdb'

Count


In [ ]:
# replace "point(\"<valueX>,<valueY>\")" with point("139.649956017232o1,35.57251104983658")
new_export_path = export_path.replace("json", "adm")
os.makedirs(os.path.dirname(new_export_path), exist_ok=True)
with open(new_export_path, "w") as f:
    for row in open(export_path, "r").readlines():
        new_row = row.replace('location":"point(\\"', 'location":point("').replace('\\")","description"', '"),"description"')
        f.write(new_row)

FileNotFoundError: [Errno 2] No such file or directory: './data/adm/cemetery.adm'